In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import json

In [4]:
##FFmpg (turns your gource into a MP4)
#brew install ffmpeg

In [5]:
##when that is finished installing type 
#brew install gource

**Run this with**:
- gource custom.log --seconds-per-day 1 --bloom-intensity 0.3
- gource custom.log --seconds-per-day 1 --bloom-intensity 0.3 -e 0.2 -i 120
- gource custom.log --seconds-per-day 1 --bloom-intensity 0.3 -e 0.2 -i 120 --title "Development of the amount of characters in Rick and Morty"
- gource custom.log --seconds-per-day 1 -1440x900 --bloom-intensity 0.1 -e 0.2 -i 120 --title "Development of the amount of characters in Rick and Morty"
- gource custom.log -1280x720 --seconds-per-day 2 --key --hide filenames

**Output video**:
- gource custom.log -1280x720 --seconds-per-day 2 --key --hide dirnames, -o - | ffmpeg -y -r 60 -f image2pipe -vcodec ppm -i - -vcodec libx264 -preset ultrafast -crf 1 -threads 0 -bf 0 output.mp4

- link : [custom log format](https://github.com/acaudwell/Gource/wiki/Custom-Log-Format)
- link : [visualization](https://edgeryders.eu/t/visualizing-our-7-years-of-online-dialogue-with-gource/11905)
- likn : [stuff to gource](https://leonjza.github.io/blog/2013/07/27/stuff-to-gource./#get-the-code-setup-your-own)

The software to create this is Gource 2, an open source tool originally meant to visualize Git repositories. Fortunately it also can read a custom log format 9.

For Discourse, the custom log file would be a text file with each line containing the following components, delimited by `|`:

- timestamp
- username
- type of action (Added, Modified, Deleted); the “M” is useful esp. for wikis
- path to the affected resource. Usually this applies to files, but in our case it would be about posts. This is the part we’d have to adapt to Discourse. Our path format could be `category/subcategory/topic-id/post-id` or it could even include the full threading information with any depth of nesting, such as `category/subcategory/topic-id/parent-post-id/parent-post-id/post-id`
- color for the affected resource (optional); this could be chosen according to the category color on edgeryders.eu

Example line:

- `1275543595|matthias|A|earthos/reef/1234/1`


In [2]:
import pandas as pd
import pandasql as ps
import numpy as np
import re
import tqdm
from tqdm import tqdm
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

In [3]:
df_characters = pd.read_csv('../df_characters.csv')
df_episodes = pd.read_csv('../df_episodes.csv')
df_transcripts = pd.read_csv('../df_transcripts.csv')
df_rel_sent = pd.read_csv('../df_rel_sent.csv')

In [4]:
from ast import literal_eval
df_characters['episode'] = df_characters['episode'].apply(literal_eval)

In [5]:
df_characters_exploded = df_characters.explode('episode')

In [6]:
df_characters_exploded.episode = df_characters_exploded.episode.astype(int)

In [7]:
df_characters_exploded

,name,status,species,type,gender,origin,location,image,episode,url,created,references
0,Rick Sanchez,Alive,Human,NaN,Male,Earth (C-137),Citadel of Ricks,https://rickandmortyapi.com/api/character/avat...,1,https://rickandmortyapi.com/api/character/1,2017-11-04T18:48:46.250Z,"['Morty Smith', 'Summer Smith', 'Beth Smith', ..."
0,Rick Sanchez,Alive,Human,NaN,Male,Earth (C-137),Citadel of Ricks,https://rickandmortyapi.com/api/character/avat...,2,https://rickandmortyapi.com/api/character/1,2017-11-04T18:48:46.250Z,"['Morty Smith', 'Summer Smith', 'Beth Smith', ..."
0,Rick Sanchez,Alive,Human,NaN,Male,Earth (C-137),Citadel of Ricks,https://rickandmortyapi.com/api/character/avat...,3,https://rickandmortyapi.com/api/character/1,2017-11-04T18:48:46.250Z,"['Morty Smith', 'Summer Smith', 'Beth Smith', ..."
0,Rick Sanchez,Alive,Human,NaN,Male,Earth (C-137),Citadel of Ricks,https://rickandmortyapi.com/api/character/avat...,4,https://rickandmortyapi.com/api/character/1,2017-11-04T18:48:46.250Z,"['Morty Smith', 'Summer Smith', 'Beth Smith', ..."
0,Rick Sanchez,Alive,Human,NaN,Male,Earth (C-137),Citadel of Ricks,https://rickandmortyapi.com/api/character/avat...,5,https://rickandmortyapi.com/api/character/1,2017-11-04T18:48:46.250Z,"['Morty Smith', 'Summer Smith', 'Beth Smith', ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
821,Young Jerry,unknown,Human,NaN,Male,Earth (Unknown dimension),Earth (Unknown dimension),https://rickandmortyapi.com/api/character/avat...,51,https://rickandmortyapi.com/api/character/822,2021-11-02T17:18:31.934Z,NaN
822,Young Beth,unknown,Human,NaN,Female,Earth (Unknown dimension),Earth (Unknown dimension),https://rickandmortyapi.com/api/character/avat...,51,https://rickandmortyapi.com/api/character/823,2021-11-02T17:19:00.951Z,NaN
823,Young Beth,unknown,Human,NaN,Female,Earth (Unknown dimension),Earth (Unknown dimension),https://rickandmortyapi.com/api/character/avat...,51,https://rickandmortyapi.com/api/character/824,2021-11-02T17:19:47.957Z,NaN
824,Young Jerry,unknown,Human,NaN,Male,Earth (Unknown dimension),Earth (Unknown dimension),https://rickandmortyapi.com/api/character/avat...,51,https://rickandmortyapi.com/api/character/825,2021-11-02T17:20:14.305Z,NaN


In [8]:
import datetime
from datetime import datetime
from datetime import timedelta
import time

In [9]:
date_range = pd.date_range(datetime.today() - timedelta(days=51), periods=51)
date_series = pd.Series(date_range)
df_episode_unix_timestamp = pd.DataFrame(pd.DataFrame(date_series).values[::-1])\
    .reset_index(drop=False)\
    .rename(columns={'index':'episode', 0:'timestamp'})
df_episode_unix_timestamp['episode'] = df_episode_unix_timestamp['episode'].apply(lambda x: x+1)
df_episode_unix_timestamp['unixtime'] = df_episode_unix_timestamp['timestamp'].astype(int)
df_episode_unix_timestamp


,episode,timestamp,unixtime
0,1,2021-12-01 11:20:31.095073,1638357631095073000
1,2,2021-11-30 11:20:31.095073,1638271231095073000
2,3,2021-11-29 11:20:31.095073,1638184831095073000
3,4,2021-11-28 11:20:31.095073,1638098431095073000
4,5,2021-11-27 11:20:31.095073,1638012031095073000
5,6,2021-11-26 11:20:31.095073,1637925631095073000
6,7,2021-11-25 11:20:31.095073,1637839231095073000
7,8,2021-11-24 11:20:31.095073,1637752831095073000
8,9,2021-11-23 11:20:31.095073,1637666431095073000
9,10,2021-11-22 11:20:31.095073,1637580031095073000


In [10]:
date_range = pd.date_range(datetime.today() - timedelta(days=19), periods=19)
date_series = pd.Series(date_range)
df_episode_unix_timestamp = pd.DataFrame(pd.DataFrame(date_series))\
    .reset_index(drop=False)\
    .rename(columns={'index':'episode', 0:'timestamp'})
df_episode_unix_timestamp['episode'] = df_episode_unix_timestamp['episode'].apply(lambda x: x+1)
df_episode_unix_timestamp['unixtime'] = df_episode_unix_timestamp['timestamp'].astype(int)
df_episode_unix_timestamp['unixtime'] = (df_episode_unix_timestamp.timestamp - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')


In [11]:
df_episode_unix_timestamp

,episode,timestamp,unixtime
0,1,2021-11-13 11:20:32.359525,1636802432
1,2,2021-11-14 11:20:32.359525,1636888832
2,3,2021-11-15 11:20:32.359525,1636975232
3,4,2021-11-16 11:20:32.359525,1637061632
4,5,2021-11-17 11:20:32.359525,1637148032
5,6,2021-11-18 11:20:32.359525,1637234432
6,7,2021-11-19 11:20:32.359525,1637320832
7,8,2021-11-20 11:20:32.359525,1637407232
8,9,2021-11-21 11:20:32.359525,1637493632
9,10,2021-11-22 11:20:32.359525,1637580032


The software to create this is Gource 2, an open source tool originally meant to visualize Git repositories. Fortunately it also can read a custom log format 9.

For Discourse, the custom log file would be a text file with each line containing the following components, delimited by `|`:

- timestamp
- username
- type of action (Added, Modified, Deleted); the “M” is useful esp. for wikis
- path to the affected resource. Usually this applies to files, but in our case it would be about posts. This is the part we’d have to adapt to Discourse. Our path format could be `category/subcategory/topic-id/post-id` or it could even include the full threading information with any depth of nesting, such as `category/subcategory/topic-id/parent-post-id/parent-post-id/post-id`
- color for the affected resource (optional); this could be chosen according to the category color on edgeryders.eu

Example line:

- `1275543595|matthias|A|earthos/reef/1234/1`


In [12]:
#df_characters_exploded

# Character pr. episode (custom_char_episode.log)

- 1633360391|episode_1|A|episode_1/rick/rick.cpp
- 1633360391|episode_1|A|episode_1/morty/rick.cpp
- 1633360391|episode_1|A|episode_1/summer/rick.cpp
- 1633360391|episode_2|A|episode_2/rick.cpp
- 1633360391|episode_3|A|episode_3/rick.cpp

In [13]:
res = ps.sqldf(
    """
    select 
        1633360391 || '|' || 'episode_' || episode || '|' || 'A' || '|' || 'episode_' || episode || '/' || character || '/' || 'player.cpp' as output
    from
    
    (
    
    select 
       episode
    ,  character
    from 
        (

        select
              replace(df.name, ' ', '_') as character
            , df.episode
        from df_characters_exploded df
        order by df.episode asc
        )
    order by episode asc
    )
    """
)

res

,output
0,1633360391|episode_1|A|episode_1/Rick_Sanchez/...
1,1633360391|episode_1|A|episode_1/Morty_Smith/p...
2,1633360391|episode_1|A|episode_1/Bepisian/play...
3,1633360391|episode_1|A|episode_1/Beth_Smith/pl...
4,1633360391|episode_1|A|episode_1/Canklanker_Th...
...,...
1262,1633360391|episode_51|A|episode_51/Redhead_Mor...
1263,1633360391|episode_51|A|episode_51/Young_Jerry...
1264,1633360391|episode_51|A|episode_51/Young_Beth/...
1265,1633360391|episode_51|A|episode_51/Young_Beth/...


In [14]:
with open('custom_char_episode.log', 'w') as f:
    for row in res.output:
        #print(row)
        f.write(row + '\n')

# Episode, Sentiment pr Character (top 5 talking pr episode)

The idea about this gource is to:
- Episodes are branches to eachother (linked)
- For each episode, take the top 5 (total sentences) characters
- Color the character according to sentiment

In [15]:
#df_rel_sent

In [108]:
#df_episodes

In [109]:
temp_df = ps.sqldf(
    """
    
    select 
        episode
      , alias as character
      , compound as sentiment
      , rn
    from
    (
    select 
      df.*
    , row_number() over (partition by episode order by num_sentences desc) as rn
    from df_rel_sent as df
    )
    
    where rn <= 5
    """
)

- `1275543595|matthias|A|earthos/reef/1234/1`

In [110]:
temp_df.head(10)

,episode,character,sentiment,rn
0,Anatomy Park,Rick,0.8118,1
1,Anatomy Park,Morty,-0.5759,2
2,Anatomy Park,Jerry,0.9647,3
3,Anatomy Park,Annie,0.8002,4
4,Anatomy Park,Dr. Bloom,0.0000,5
5,Auto Erotic Assimilation,Rick,0.5279,1
6,Gotron Jerrysis Rickvangelion,Rick,0.9944,1
7,Gotron Jerrysis Rickvangelion,Summer,0.9942,2
8,Gotron Jerrysis Rickvangelion,Morty,0.6733,3
9,Gotron Jerrysis Rickvangelion,Beth,0.8852,4


In [111]:
def rgb(minimum, maximum, value):
    minimum, maximum = float(minimum), float(maximum)
    ratio = 2 * (value-minimum) / (maximum - minimum)
    b = int(max(0, 255*(1 - ratio)))
    r = int(max(0, 255*(ratio - 1)))
    g = 0#255 - b - r
    return r, g, b


def rgb_to_hex(rgb):
    return '%02x%02x%02x' % rgb

In [112]:
rgb_to_hex(rgb(1, -1, -1))

'ff0000'

In [113]:
# As we have 19 episodes
df_episode_unix_timestamp

,episode,timestamp,unixtime
0,1,2021-10-18 21:59:55.073484,1634594395
1,2,2021-10-19 21:59:55.073484,1634680795
2,3,2021-10-20 21:59:55.073484,1634767195
3,4,2021-10-21 21:59:55.073484,1634853595
4,5,2021-10-22 21:59:55.073484,1634939995
5,6,2021-10-23 21:59:55.073484,1635026395
6,7,2021-10-24 21:59:55.073484,1635112795
7,8,2021-10-25 21:59:55.073484,1635199195
8,9,2021-10-26 21:59:55.073484,1635285595
9,10,2021-10-27 21:59:55.073484,1635371995


In [114]:
res = pd.merge(
    temp_df,
    df_episodes[['name', 'id']],
    how='left',
    left_on='episode',
    right_on='name'
).sort_values(['id', 'rn'], ascending=True).reset_index(drop=True)
res

,episode,character,sentiment,rn,name,id
0,Pilot,Rick,0.9975,1,Pilot,1
1,Pilot,Morty,0.9971,2,Pilot,1
2,Pilot,Jerry,0.9619,3,Pilot,1
3,Pilot,Beth,0.8590,4,Pilot,1
4,Pilot,Mr. Goldenfold,0.6080,5,Pilot,1
...,...,...,...,...,...,...
106,Rickmurai Jack,Rick,0.9806,1,Rickmurai Jack,51
107,Rickmurai Jack,Morty,0.9892,2,Rickmurai Jack,51
108,Rickmurai Jack,President Morty,0.8430,3,Rickmurai Jack,51
109,Rickmurai Jack,Crowscare,0.9445,4,Rickmurai Jack,51


In [115]:
res[res.episode =='Anatomy Park']

,episode,character,sentiment,rn,name,id
10,Anatomy Park,Rick,0.8118,1,Anatomy Park,3
11,Anatomy Park,Morty,-0.5759,2,Anatomy Park,3
12,Anatomy Park,Jerry,0.9647,3,Anatomy Park,3
13,Anatomy Park,Annie,0.8002,4,Anatomy Park,3
14,Anatomy Park,Dr. Bloom,0.0000,5,Anatomy Park,3


In [116]:
with open('episode_character_sentiment.log', 'w+') as f:
        

    n = 0
    for idx, row in res.iterrows():
        #print('1633360391' + '|' + str(row.episode))
        if idx == 0:
            episode = res.iloc[idx].episode
            episode_gathered = res.iloc[idx].episode
            folder = episode_gathered + '/'
            #print('idx0 :', episode)
            #print(res[res.episode == episode].character)
            #print(timestamp + '|' + 'sentiment|A|' + folder)
            timestamp = str(df_episode_unix_timestamp.iloc[n].unixtime)
            f.write(timestamp + '|' + 'sentiment|A|' + folder + '\n')
            for char in res[res.episode == episode].character:
                #print(episode + '/' + char + '.txt')
                #print(res[(res.episode == episode) & (res.character == char)].sentiment.values[0])
                sentiment_score = res[(res.episode == episode) & (res.character == char)].sentiment.values[0]
                sentiment_to_hex = rgb_to_hex(rgb(1, -1, sentiment_score))
                #print(sentiment_to_hex)
                #print(episode + '/' + char + '.txt' + '|' + sentiment_to_hex.upper())
                add_file = episode_gathered + '/' + char + '|' + sentiment_to_hex.upper()
                print(timestamp + '|' + 'sentiment|A|' + add_file)
                f.write(timestamp + '|' + 'sentiment|A|' + add_file + '\n')
                
        if idx != 0:
            episode = res.iloc[idx].episode 
            #for char in res[res.episode == episode].character:
            #    print(char)
            if episode != res.iloc[idx - 1].episode:
                episode = res.iloc[idx].episode
                if n == 0:
                    n += 1
                    continue
                try:
                    timestamp = str(df_episode_unix_timestamp.iloc[n].unixtime)
                except:
                    None
                #print('idx :', episode, res.iloc[idx].episode)
                episode_gathered += '/' + res.iloc[idx].episode
                if n % 3 == 0:
                    episode_gathered = res.iloc[idx].episode
                folder = episode_gathered + '/'
                print(timestamp + '|' + 'sentiment|A|' + folder)
                f.write(timestamp + '|' + 'sentiment|A|' + folder + '\n')
                #print('gathered : ', episode_gathered)
                for char in res[res.episode == episode].character:
                    sentiment_score = res[(res.episode == episode) & (res.character == char)].sentiment.values[0]
                    sentiment_to_hex = rgb_to_hex(rgb(1, -1, sentiment_score))
                    add_file = episode_gathered + '/' + char + '|' + sentiment_to_hex.upper()
                    print(timestamp + '|' + 'sentiment|A|' + add_file)
                    f.write(timestamp + '|' + 'sentiment|A|' + add_file + '\n')
                    
                n += 1
                #print(folder + '\n')
                #print('now : ', res.iloc[idx].episode)
                #print('before : ', res.iloc[idx-1].episode)
                #print(episode)
                #break
                

1634594395|sentiment|A|Pilot/Rick|0000FE
1634594395|sentiment|A|Pilot/Morty|0000FE
1634594395|sentiment|A|Pilot/Jerry|0000F5
1634594395|sentiment|A|Pilot/Beth|0000DB
1634594395|sentiment|A|Pilot/Mr. Goldenfold|00009B
1634680795|sentiment|A|Pilot/Anatomy Park/
1634680795|sentiment|A|Pilot/Anatomy Park/Rick|0000CF
1634680795|sentiment|A|Pilot/Anatomy Park/Morty|920000
1634680795|sentiment|A|Pilot/Anatomy Park/Jerry|0000F5
1634680795|sentiment|A|Pilot/Anatomy Park/Annie|0000CC
1634680795|sentiment|A|Pilot/Anatomy Park/Dr. Bloom|000000
1634767195|sentiment|A|Pilot/Anatomy Park/M. Night Shaym-Aliens!/
1634767195|sentiment|A|Pilot/Anatomy Park/M. Night Shaym-Aliens!/Rick|0000FE
1634767195|sentiment|A|Pilot/Anatomy Park/M. Night Shaym-Aliens!/Morty|0000E8
1634767195|sentiment|A|Pilot/Anatomy Park/M. Night Shaym-Aliens!/Jerry|0000FB
1634767195|sentiment|A|Pilot/Anatomy Park/M. Night Shaym-Aliens!/Prince Nebulon|0000FC
1634767195|sentiment|A|Pilot/Anatomy Park/M. Night Shaym-Aliens!/Mr. Marklev

**Note:**
- Run with:
- gource episode_character_sentiment.log --background-image /Users/christianr/Documents/DTU/1_semester/02805_Social_graphs_and_interactions/project/wcimages/graph_background.png -1280x720 --seconds-per-day 2 --key --filename-time 50 -o - | ffmpeg -y -r 60 -f image2pipe -vcodec ppm -i - -vcodec libx264 -preset ultrafast -crf 1 -threads 0 -bf 0 episode_character_sentiment.mp4

# Episode, Sentiment pr Character (Main Characters pr. episode)

The idea about this gource is to:
- Episodes are branches to eachother (linked)
- For each episode, take the top 5 (total sentences) characters
- Color the character according to sentiment

In [117]:
#df_rel_sent

In [118]:
#df_episodes

In [119]:
temp_df = ps.sqldf(
    """
    
    select 
        episode
      , alias as character
      , compound as sentiment
      , rn
    from
    (
    select 
      df.*
    , row_number() over (partition by episode order by num_sentences desc) as rn
    from df_rel_sent as df
    )
    where lower(character) like '%rick%'
       or lower(character) like '%morty%'
       or lower(character) like '%summer%'
       or lower(character) like '%jerry%'
       or lower(character) like '%beth%'
    --where rn <= 5
    """
)
temp_df

,episode,character,sentiment,rn
0,Anatomy Park,Rick,0.8118,1
1,Anatomy Park,Morty,-0.5759,2
2,Anatomy Park,Jerry,0.9647,3
3,Anatomy Park,Beth,0.6852,7
4,Anatomy Park,Summer,-0.5000,9
...,...,...,...,...
120,Vindicators 3: The Return of Worldender,Rick,0.9959,1
121,Vindicators 3: The Return of Worldender,Morty,-0.9213,2
122,Vindicators 3: The Return of Worldender,Drunk Rick,-0.9381,4
123,Vindicators 3: The Return of Worldender,Beth,0.0000,7


- `1275543595|matthias|A|earthos/reef/1234/1`

In [120]:
temp_df.head(10)

,episode,character,sentiment,rn
0,Anatomy Park,Rick,0.8118,1
1,Anatomy Park,Morty,-0.5759,2
2,Anatomy Park,Jerry,0.9647,3
3,Anatomy Park,Beth,0.6852,7
4,Anatomy Park,Summer,-0.5000,9
5,Auto Erotic Assimilation,Rick,0.5279,1
6,Gotron Jerrysis Rickvangelion,Rick,0.9944,1
7,Gotron Jerrysis Rickvangelion,Summer,0.9942,2
8,Gotron Jerrysis Rickvangelion,Morty,0.6733,3
9,Gotron Jerrysis Rickvangelion,Beth,0.8852,4


In [121]:
def rgb(minimum, maximum, value):
    minimum, maximum = float(minimum), float(maximum)
    ratio = 2 * (value-minimum) / (maximum - minimum)
    b = int(max(0, 255*(1 - ratio)))
    r = int(max(0, 255*(ratio - 1)))
    g = 0#255 - b - r
    return r, g, b


def rgb_to_hex(rgb):
    return '%02x%02x%02x' % rgb

In [122]:
rgb_to_hex(rgb(1, -1, -1))

'ff0000'

In [123]:
# As we have 19 episodes
df_episode_unix_timestamp

,episode,timestamp,unixtime
0,1,2021-10-18 21:59:55.073484,1634594395
1,2,2021-10-19 21:59:55.073484,1634680795
2,3,2021-10-20 21:59:55.073484,1634767195
3,4,2021-10-21 21:59:55.073484,1634853595
4,5,2021-10-22 21:59:55.073484,1634939995
5,6,2021-10-23 21:59:55.073484,1635026395
6,7,2021-10-24 21:59:55.073484,1635112795
7,8,2021-10-25 21:59:55.073484,1635199195
8,9,2021-10-26 21:59:55.073484,1635285595
9,10,2021-10-27 21:59:55.073484,1635371995


In [124]:
res = pd.merge(
    temp_df,
    df_episodes[['name', 'id']],
    how='left',
    left_on='episode',
    right_on='name'
).sort_values(['id', 'rn'], ascending=True).reset_index(drop=True)
res

,episode,character,sentiment,rn,name,id
0,Pilot,Rick,0.9975,1,Pilot,1
1,Pilot,Morty,0.9971,2,Pilot,1
2,Pilot,Jerry,0.9619,3,Pilot,1
3,Pilot,Beth,0.8590,4,Pilot,1
4,Pilot,Summer,-0.7716,6,Pilot,1
...,...,...,...,...,...,...
120,Rickmurai Jack,Morty,0.9892,2,Rickmurai Jack,51
121,Rickmurai Jack,President Morty,0.8430,3,Rickmurai Jack,51
122,Rickmurai Jack,Jerry,0.3612,7,Rickmurai Jack,51
123,Rickmurai Jack,Beth,0.7646,8,Rickmurai Jack,51


In [125]:
res[res.episode =='Rickmurai Jack']

,episode,character,sentiment,rn,name,id
119,Rickmurai Jack,Rick,0.9806,1,Rickmurai Jack,51
120,Rickmurai Jack,Morty,0.9892,2,Rickmurai Jack,51
121,Rickmurai Jack,President Morty,0.8430,3,Rickmurai Jack,51
122,Rickmurai Jack,Jerry,0.3612,7,Rickmurai Jack,51
123,Rickmurai Jack,Beth,0.7646,8,Rickmurai Jack,51
124,Rickmurai Jack,Summer,0.6369,9,Rickmurai Jack,51


In [126]:
with open('episode_main_character_sentiment.log', 'w+') as f:
        

    n = 0
    for idx, row in res.iterrows():
        #print('1633360391' + '|' + str(row.episode))
        if idx == 0:
            episode = res.iloc[idx].episode
            episode_gathered = res.iloc[idx].episode
            folder = episode_gathered + '/'
            #print('idx0 :', episode)
            #print(res[res.episode == episode].character)
            #print(timestamp + '|' + 'sentiment|A|' + folder)
            f.write(timestamp + '|' + 'sentiment|A|' + folder + '\n')
            timestamp = str(df_episode_unix_timestamp.iloc[n].unixtime)
            for char in res[res.episode == episode].character:
                #print(episode + '/' + char + '.txt')
                #print(res[(res.episode == episode) & (res.character == char)].sentiment.values[0])
                sentiment_score = res[(res.episode == episode) & (res.character == char)].sentiment.values[0]
                sentiment_to_hex = rgb_to_hex(rgb(1, -1, sentiment_score))
                #print(sentiment_to_hex)
                #print(episode + '/' + char + '.txt' + '|' + sentiment_to_hex.upper())
                add_file = episode_gathered + '/' + char + '|' + sentiment_to_hex.upper()
                print(timestamp + '|' + 'sentiment|A|' + add_file)
                f.write(timestamp + '|' + 'sentiment|A|' + add_file + '\n')
                
        if idx != 0:
            episode = res.iloc[idx].episode 
            #for char in res[res.episode == episode].character:
            #    print(char)
            if episode != res.iloc[idx - 1].episode:
                episode = res.iloc[idx].episode
                if n == 0:
                    n += 1
                    continue
                try:
                    timestamp = str(df_episode_unix_timestamp.iloc[n].unixtime)
                except:
                    None
                #print('idx :', episode, res.iloc[idx].episode)
                episode_gathered += '/' + res.iloc[idx].episode
                if n % 3 == 0:
                    episode_gathered = res.iloc[idx].episode
                folder = episode_gathered + '/'
                print(timestamp + '|' + 'sentiment|A|' + folder)
                f.write(timestamp + '|' + 'sentiment|A|' + folder + '\n')
                #print('gathered : ', episode_gathered)
                for char in res[res.episode == episode].character:
                    sentiment_score = res[(res.episode == episode) & (res.character == char)].sentiment.values[0]
                    sentiment_to_hex = rgb_to_hex(rgb(1, -1, sentiment_score))
                    add_file = episode_gathered + '/' + char + '|' + sentiment_to_hex.upper()
                    print(timestamp + '|' + 'sentiment|A|' + add_file)
                    f.write(timestamp + '|' + 'sentiment|A|' + add_file + '\n')
                    
                n += 1
                #print(folder + '\n')
                #print('now : ', res.iloc[idx].episode)
                #print('before : ', res.iloc[idx-1].episode)
                #print(episode)
                #break
                

1634594395|sentiment|A|Pilot/Rick|0000FE
1634594395|sentiment|A|Pilot/Morty|0000FE
1634594395|sentiment|A|Pilot/Jerry|0000F5
1634594395|sentiment|A|Pilot/Beth|0000DB
1634594395|sentiment|A|Pilot/Summer|C40000
1634680795|sentiment|A|Pilot/Anatomy Park/
1634680795|sentiment|A|Pilot/Anatomy Park/Rick|0000CF
1634680795|sentiment|A|Pilot/Anatomy Park/Morty|920000
1634680795|sentiment|A|Pilot/Anatomy Park/Jerry|0000F5
1634680795|sentiment|A|Pilot/Anatomy Park/Beth|0000AE
1634680795|sentiment|A|Pilot/Anatomy Park/Summer|7F0000
1634767195|sentiment|A|Pilot/Anatomy Park/M. Night Shaym-Aliens!/
1634767195|sentiment|A|Pilot/Anatomy Park/M. Night Shaym-Aliens!/Rick|0000FE
1634767195|sentiment|A|Pilot/Anatomy Park/M. Night Shaym-Aliens!/Morty|0000E8
1634767195|sentiment|A|Pilot/Anatomy Park/M. Night Shaym-Aliens!/Jerry|0000FB
1634767195|sentiment|A|Pilot/Anatomy Park/M. Night Shaym-Aliens!/Beth|0000B2
1634853595|sentiment|A|Meeseeks and Destroy/
1634853595|sentiment|A|Meeseeks and Destroy/Rick|0000

**Note:**
- Run with:
- gource custom.log --background-image /Users/christianr/Documents/DTU/1_semester/02805_Social_graphs_and_interactions/project/wcimages/graph_background.png -1280x720 --seconds-per-day 2 --key --hide dirnames, -o - | ffmpeg -y -r 60 -f image2pipe -vcodec ppm -i - -vcodec libx264 -preset ultrafast -crf 1 -threads 0 -bf 0 output.mp4

THIS WORKS
- gource custom.log --background-image /Users/christianr/Documents/DTU/1_semester/02805_Social_graphs_and_interactions/project/wcimages/graph_background.png -1280x720 --seconds-per-day 2 --key --highlight-dirs -o - | ffmpeg -y -r 60 -f image2pipe -vcodec ppm -i - -vcodec libx264 -preset ultrafast -crf 1 -threads 0 -bf 0 output.mp4


- gource episode_main_character_sentiment.log --background-image /Users/christianr/Documents/DTU/1_semester/02805_Social_graphs_and_interactions/project/wcimages/graph_background.png -1280x720 --seconds-per-day 2 --key --highlight-dirs -o - | ffmpeg -y -r 60 -f image2pipe -vcodec ppm -i - -vcodec libx264 -preset ultrafast -crf 1 -threads 0 -bf 0 episode_main_character_sentiment.mp4

# Jerry

In [13]:
temp_df = ps.sqldf(
    """
    with jerry_episodes as (
    select 
        'Jerry' as char_name
      ,  episode
    from df_characters_exploded
    where lower(name) like '%jerry%'
    group by 1, 2
    order by episode
    )
    
    select 
        episode as episode_nr
      , name as character
    from df_characters_exploded df_ex
    where df_ex.episode in (select distinct episode from jerry_episodes)
    and lower(name) not like '%jerry%'
    group by 1, 2
    order by 1, 2
    """
)
temp_df

,episode_nr,character
0,1,Bepisian
1,1,Beth Smith
2,1,Canklanker Thom
3,1,Davin
4,1,Frank Palicky
...,...,...
929,51,Summer Smith
930,51,Two Crows
931,51,Unknown Rick
932,51,Young Beth


In [14]:
res = pd.merge(
    temp_df,
    df_episodes[['name', 'id']],
    how='left',
    left_on='episode_nr',
    right_on='id'
).sort_values(['id'], ascending=True).reset_index(drop=True).rename(columns={'name':'episode'})
res

,episode_nr,character,episode,id
0,1,Bepisian,Pilot,1
1,1,Summer Smith,Pilot,1
2,1,Rick Sanchez,Pilot,1
3,1,Pripudlian,Pilot,1
4,1,Principal Vagina,Pilot,1
...,...,...,...,...
929,51,Diane Sanchez,Rickmurai Jack,51
930,51,Evil Morty,Rickmurai Jack,51
931,51,Long Hair Rick,Rickmurai Jack,51
932,51,Beth Smith,Rickmurai Jack,51


In [15]:
n_days = len(res.episode.unique().tolist())

In [16]:
date_range = pd.date_range(datetime.today() - timedelta(days=n_days), periods=n_days)
date_series = pd.Series(date_range)
df_episode_unix_timestamp = pd.DataFrame(pd.DataFrame(date_series))\
    .reset_index(drop=False)\
    .rename(columns={'index':'episode', 0:'timestamp'})
df_episode_unix_timestamp['episode'] = df_episode_unix_timestamp['episode'].apply(lambda x: x+1)
df_episode_unix_timestamp['unixtime'] = df_episode_unix_timestamp['timestamp'].astype(int)
df_episode_unix_timestamp['unixtime'] = (df_episode_unix_timestamp.timestamp - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
#df_episode_unix_timestamp

In [17]:
#res

In [18]:
with open('jerry_through_episodes.log', 'w+') as f:
    n = 0
    for idx, row in res.iterrows():
        #print(idx)
        if idx == 0:
            episode = res.iloc[idx].episode
            episode_gathered = res.iloc[idx].episode
            folder = episode_gathered + '/'
            #print(episode)
            #print('idx0 :', episode)
            #print(res[res.episode == episode].character)
            #print(timestamp + '|' + 'sentiment|A|' + folder)
            #f.write(timestamp + '|' + 'sentiment|A|' + folder + '\n')
            timestamp = str(df_episode_unix_timestamp.iloc[n].unixtime)
            #print(timestamp)
            for char in res[res.episode == episode].character:
                #print(episode + '/' + char)
                print(timestamp + '|' + 'Jerry' + '|' + 'A' + '|' + episode + '/' + char)
                f.write(timestamp + '|' + 'Jerry' + '|' + 'A' + '|' + episode + '/' + char + '\n')


        if idx != 0:
            episode = res.iloc[idx].episode 
            #for char in res[res.episode == episode].character:
            #    print(char)
            if episode != res.iloc[idx - 1].episode:
                episode = res.iloc[idx].episode
                #print(episode)

                if n == 0:
                    n += 1
                    continue
                try:
                    timestamp = str(df_episode_unix_timestamp.iloc[n].unixtime)
                except:
                    None
                #print('idx :', episode, res.iloc[idx].episode)
                episode_gathered += '/' + res.iloc[idx].episode
                #print(episode_gathered)
                if n % 4 == 0:
                    episode_gathered = res.iloc[idx].episode
                folder = episode_gathered + '/'
                print(timestamp + '|' + 'Jerry|A|' + folder)
                #f.write(timestamp + '|' + 'sentiment|A|' + folder + '\n')
                #print('gathered : ', episode_gathered)
                for char in res[res.episode == episode].character:
                    #print(timestamp + '|' + 'sentiment|A|' + add_file)
                    print(timestamp + '|' + 'Jerry' + '|' + 'A' + '|' + episode_gathered + '/' + char)
                    f.write(timestamp + '|' + 'Jerry' + '|' + 'A' + '|' + episode_gathered + '/' + char + '\n')

                n += 1
                #print(folder + '\n')
                #print('now : ', res.iloc[idx].episode)
                #print('before : ', res.iloc[idx-1].episode)
                #print(episode)


1634642449|Jerry|A|Pilot/Bepisian
1634642449|Jerry|A|Pilot/Summer Smith
1634642449|Jerry|A|Pilot/Rick Sanchez
1634642449|Jerry|A|Pilot/Pripudlian
1634642449|Jerry|A|Pilot/Principal Vagina
1634642449|Jerry|A|Pilot/Mr. Goldenfold
1634642449|Jerry|A|Pilot/Morty Smith
1634642449|Jerry|A|Pilot/Jessica's Friend
1634642449|Jerry|A|Pilot/Mrs. Sanchez
1634642449|Jerry|A|Pilot/Hookah Alien
1634642449|Jerry|A|Pilot/Greebybobe
1634642449|Jerry|A|Pilot/Glenn
1634642449|Jerry|A|Pilot/Frank Palicky
1634642449|Jerry|A|Pilot/Davin
1634642449|Jerry|A|Pilot/Canklanker Thom
1634642449|Jerry|A|Pilot/Beth Smith
1634642449|Jerry|A|Pilot/Jessica
1634728849|Jerry|A|Pilot/Anatomy Park/
1634728849|Jerry|A|Pilot/Anatomy Park/Joyce Smith
1634728849|Jerry|A|Pilot/Anatomy Park/Leonard Smith
1634728849|Jerry|A|Pilot/Anatomy Park/Morty Smith
1634728849|Jerry|A|Pilot/Anatomy Park/Poncho
1634728849|Jerry|A|Pilot/Anatomy Park/Tom Randolph
1634728849|Jerry|A|Pilot/Anatomy Park/Roger
1634728849|Jerry|A|Pilot/Anatomy Park/R

1636716049|Jerry|A|The Rickchurian Mortydate/
1636716049|Jerry|A|The Rickchurian Mortydate/Summer Smith
1636716049|Jerry|A|The Rickchurian Mortydate/Steve
1636716049|Jerry|A|The Rickchurian Mortydate/Rick Sanchez
1636716049|Jerry|A|The Rickchurian Mortydate/Presidentress of The Mega Gargantuans
1636716049|Jerry|A|The Rickchurian Mortydate/President Curtis
1636716049|Jerry|A|The Rickchurian Mortydate/Mr. Poopybutthole
1636716049|Jerry|A|The Rickchurian Mortydate/Secretary of the Interior
1636716049|Jerry|A|The Rickchurian Mortydate/Invisi-trooper
1636716049|Jerry|A|The Rickchurian Mortydate/Beth Smith
1636716049|Jerry|A|The Rickchurian Mortydate/Baby Poopybutthole
1636716049|Jerry|A|The Rickchurian Mortydate/Amy Poopybutthole
1636716049|Jerry|A|The Rickchurian Mortydate/Alien Googah
1636716049|Jerry|A|The Rickchurian Mortydate/Morty Smith
1636802449|Jerry|A|The Rickchurian Mortydate/Edge of Tomorty: Rick, Die, Rickpeat/
1636802449|Jerry|A|The Rickchurian Mortydate/Edge of Tomorty: Rick,

**Note:**
- Run with
- gource jerry_through_episodes.log --background-image /Users/christianr/Documents/DTU/1_semester/02805_Social_graphs_and_interactions/project/wcimages/jerry_background.png -1280x720 --seconds-per-day 2 --key --highlight-dirs

- gource jerry_through_episodes.log --background-image /Users/christianr/Documents/DTU/1_semester/02805_Social_graphs_and_interactions/project/wcimages/jerry_background.png -1280x720 --seconds-per-day 2 --key --highlight-dirs --highlight-user Jerry --filename-time 4

# Beth

In [171]:
temp_df = ps.sqldf(
    """
    with jerry_episodes as (
    select 
        'Beth' as char_name
      ,  episode
    from df_characters_exploded
    where lower(name) like '%beth%'
    group by 1, 2
    order by episode
    )
    
    select 
        episode as episode_nr
      , name as character
    from df_characters_exploded df_ex
    where df_ex.episode in (select distinct episode from jerry_episodes)
    and lower(name) not like '%beth%'
    group by 1, 2
    order by 1, 2
    """
)
temp_df

,episode_nr,character
0,1,Bepisian
1,1,Canklanker Thom
2,1,Davin
3,1,Frank Palicky
4,1,Glenn
...,...,...
1084,51,Summer Smith
1085,51,Two Crows
1086,51,Unknown Rick
1087,51,Young Jerry


In [172]:
res = pd.merge(
    temp_df,
    df_episodes[['name', 'id']],
    how='left',
    left_on='episode_nr',
    right_on='id'
).sort_values(['id'], ascending=True).reset_index(drop=True).rename(columns={'name':'episode'})
res

,episode_nr,character,episode,id
0,1,Bepisian,Pilot,1
1,1,Summer Smith,Pilot,1
2,1,Rick Sanchez,Pilot,1
3,1,Pripudlian,Pilot,1
4,1,Principal Vagina,Pilot,1
...,...,...,...,...
1084,51,Evil Morty,Rickmurai Jack,51
1085,51,Jerry Smith,Rickmurai Jack,51
1086,51,Long Hair Rick,Rickmurai Jack,51
1087,51,Metaphor for Capitalism,Rickmurai Jack,51


In [173]:
n_days = len(res.episode.unique().tolist())

In [174]:
date_range = pd.date_range(datetime.today() - timedelta(days=n_days), periods=n_days)
date_series = pd.Series(date_range)
df_episode_unix_timestamp = pd.DataFrame(pd.DataFrame(date_series))\
    .reset_index(drop=False)\
    .rename(columns={'index':'episode', 0:'timestamp'})
df_episode_unix_timestamp['episode'] = df_episode_unix_timestamp['episode'].apply(lambda x: x+1)
df_episode_unix_timestamp['unixtime'] = df_episode_unix_timestamp['timestamp'].astype(int)
df_episode_unix_timestamp['unixtime'] = (df_episode_unix_timestamp.timestamp - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
#df_episode_unix_timestamp

In [175]:
#res

In [176]:
with open('beth_through_episodes.log', 'w+') as f:
    n = 0
    for idx, row in res.iterrows():
        #print(idx)
        if idx == 0:
            episode = res.iloc[idx].episode
            episode_gathered = res.iloc[idx].episode
            folder = episode_gathered + '/'
            #print(episode)
            #print('idx0 :', episode)
            #print(res[res.episode == episode].character)
            #print(timestamp + '|' + 'sentiment|A|' + folder)
            #f.write(timestamp + '|' + 'sentiment|A|' + folder + '\n')
            timestamp = str(df_episode_unix_timestamp.iloc[n].unixtime)
            #print(timestamp)
            for char in res[res.episode == episode].character:
                #print(episode + '/' + char)
                print(timestamp + '|' + 'Beth' + '|' + 'A' + '|' + episode + '/' + char)
                f.write(timestamp + '|' + 'Beth' + '|' + 'A' + '|' + episode + '/' + char + '\n')


        if idx != 0:
            episode = res.iloc[idx].episode 
            #for char in res[res.episode == episode].character:
            #    print(char)
            if episode != res.iloc[idx - 1].episode:
                episode = res.iloc[idx].episode
                #print(episode)

                if n == 0:
                    n += 1
                    continue
                try:
                    timestamp = str(df_episode_unix_timestamp.iloc[n].unixtime)
                except:
                    None
                #print('idx :', episode, res.iloc[idx].episode)
                episode_gathered += '/' + res.iloc[idx].episode
                #print(episode_gathered)
                if n % 3 == 0:
                    episode_gathered = res.iloc[idx].episode
                folder = episode_gathered + '/'
                print(timestamp + '|' + 'Beth|A|' + folder)
                #f.write(timestamp + '|' + 'sentiment|A|' + folder + '\n')
                #print('gathered : ', episode_gathered)
                for char in res[res.episode == episode].character:
                    #print(timestamp + '|' + 'sentiment|A|' + add_file)
                    print(timestamp + '|' + 'Beth' + '|' + 'A' + '|' + episode_gathered + '/' + char)
                    f.write(timestamp + '|' + 'Beth' + '|' + 'A' + '|' + episode_gathered + '/' + char + '\n')

                n += 1
                #print(folder + '\n')
                #print('now : ', res.iloc[idx].episode)
                #print('before : ', res.iloc[idx-1].episode)
                #print(episode)


1634251822|Beth|A|Pilot/Bepisian
1634251822|Beth|A|Pilot/Summer Smith
1634251822|Beth|A|Pilot/Rick Sanchez
1634251822|Beth|A|Pilot/Pripudlian
1634251822|Beth|A|Pilot/Principal Vagina
1634251822|Beth|A|Pilot/Mr. Goldenfold
1634251822|Beth|A|Pilot/Morty Smith
1634251822|Beth|A|Pilot/Jessica's Friend
1634251822|Beth|A|Pilot/Mrs. Sanchez
1634251822|Beth|A|Pilot/Jerry Smith
1634251822|Beth|A|Pilot/Hookah Alien
1634251822|Beth|A|Pilot/Greebybobe
1634251822|Beth|A|Pilot/Glenn
1634251822|Beth|A|Pilot/Frank Palicky
1634251822|Beth|A|Pilot/Davin
1634251822|Beth|A|Pilot/Canklanker Thom
1634251822|Beth|A|Pilot/Jessica
1634338222|Beth|A|Pilot/Anatomy Park/
1634338222|Beth|A|Pilot/Anatomy Park/Leonard Smith
1634338222|Beth|A|Pilot/Anatomy Park/Morty Smith
1634338222|Beth|A|Pilot/Anatomy Park/Poncho
1634338222|Beth|A|Pilot/Anatomy Park/Rick Sanchez
1634338222|Beth|A|Pilot/Anatomy Park/Tom Randolph
1634338222|Beth|A|Pilot/Anatomy Park/Ruben
1634338222|Beth|A|Pilot/Anatomy Park/Summer Smith
1634338222|

1636930222|Beth|A|One Crew Over the Crewcoo's Morty/Claw and Hoarder: Special Ricktim's Morty/
1636930222|Beth|A|One Crew Over the Crewcoo's Morty/Claw and Hoarder: Special Ricktim's Morty/Shadow Jacker
1636930222|Beth|A|One Crew Over the Crewcoo's Morty/Claw and Hoarder: Special Ricktim's Morty/The Wizard
1636930222|Beth|A|One Crew Over the Crewcoo's Morty/Claw and Hoarder: Special Ricktim's Morty/Talking Cat
1636930222|Beth|A|One Crew Over the Crewcoo's Morty/Claw and Hoarder: Special Ricktim's Morty/Summer Smith
1636930222|Beth|A|One Crew Over the Crewcoo's Morty/Claw and Hoarder: Special Ricktim's Morty/Slut Dragon
1636930222|Beth|A|One Crew Over the Crewcoo's Morty/Claw and Hoarder: Special Ricktim's Morty/Rick Sanchez
1636930222|Beth|A|One Crew Over the Crewcoo's Morty/Claw and Hoarder: Special Ricktim's Morty/Balthromaw
1636930222|Beth|A|One Crew Over the Crewcoo's Morty/Claw and Hoarder: Special Ricktim's Morty/Michael
1636930222|Beth|A|One Crew Over the Crewcoo's Morty/Claw an

**Note:**
- Run with
- gource beth_through_episodes.log --background-image /Users/christianr/Documents/DTU/1_semester/02805_Social_graphs_and_interactions/project/wcimages/beth_background.png -1280x720 --seconds-per-day 2 --key --highlight-dirs

- gource beth_through_episodes.log --background-image /Users/christianr/Documents/DTU/1_semester/02805_Social_graphs_and_interactions/project/wcimages/beth_background.png -1280x720 --seconds-per-day 2 --key --highlight-dirs --highlight-user Beth --filename-time 4



# Rick

In [177]:
temp_df = ps.sqldf(
    """
    with jerry_episodes as (
    select 
        'Rick' as char_name
      ,  episode
    from df_characters_exploded
    where lower(name) like '%rick%'
    group by 1, 2
    order by episode
    )
    
    select 
        episode as episode_nr
      , name as character
    from df_characters_exploded df_ex
    where df_ex.episode in (select distinct episode from jerry_episodes)
    and lower(name) not like '%rick%'
    group by 1, 2
    order by 1, 2
    """
)
temp_df

,episode_nr,character
0,1,Bepisian
1,1,Beth Smith
2,1,Canklanker Thom
3,1,Davin
4,1,Frank Palicky
...,...,...
1040,51,Redhead Morty
1041,51,Summer Smith
1042,51,Two Crows
1043,51,Young Beth


In [178]:
res = pd.merge(
    temp_df,
    df_episodes[['name', 'id']],
    how='left',
    left_on='episode_nr',
    right_on='id'
).sort_values(['id'], ascending=True).reset_index(drop=True).rename(columns={'name':'episode'})
res

,episode_nr,character,episode,id
0,1,Bepisian,Pilot,1
1,1,Summer Smith,Pilot,1
2,1,Pripudlian,Pilot,1
3,1,Principal Vagina,Pilot,1
4,1,Mrs. Sanchez,Pilot,1
...,...,...,...,...
1040,51,40 Years Old Morty,Rickmurai Jack,51
1041,51,26 Years Old Morty,Rickmurai Jack,51
1042,51,Young Beth,Rickmurai Jack,51
1043,51,Deformed Morty,Rickmurai Jack,51


In [179]:
n_days = len(res.episode.unique().tolist())

In [180]:
date_range = pd.date_range(datetime.today() - timedelta(days=n_days), periods=n_days)
date_series = pd.Series(date_range)
df_episode_unix_timestamp = pd.DataFrame(pd.DataFrame(date_series))\
    .reset_index(drop=False)\
    .rename(columns={'index':'episode', 0:'timestamp'})
df_episode_unix_timestamp['episode'] = df_episode_unix_timestamp['episode'].apply(lambda x: x+1)
df_episode_unix_timestamp['unixtime'] = df_episode_unix_timestamp['timestamp'].astype(int)
df_episode_unix_timestamp['unixtime'] = (df_episode_unix_timestamp.timestamp - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
#df_episode_unix_timestamp

In [181]:
#res

In [182]:
with open('rick_through_episodes.log', 'w+') as f:
    n = 0
    for idx, row in res.iterrows():
        #print(idx)
        if idx == 0:
            episode = res.iloc[idx].episode
            episode_gathered = res.iloc[idx].episode
            folder = episode_gathered + '/'
            #print(episode)
            #print('idx0 :', episode)
            #print(res[res.episode == episode].character)
            #print(timestamp + '|' + 'sentiment|A|' + folder)
            #f.write(timestamp + '|' + 'sentiment|A|' + folder + '\n')
            timestamp = str(df_episode_unix_timestamp.iloc[n].unixtime)
            #print(timestamp)
            for char in res[res.episode == episode].character:
                #print(episode + '/' + char)
                print(timestamp + '|' + 'Rick' + '|' + 'A' + '|' + episode + '/' + char)
                f.write(timestamp + '|' + 'Rick' + '|' + 'A' + '|' + episode + '/' + char + '\n')


        if idx != 0:
            episode = res.iloc[idx].episode 
            #for char in res[res.episode == episode].character:
            #    print(char)
            if episode != res.iloc[idx - 1].episode:
                episode = res.iloc[idx].episode
                #print(episode)

                if n == 0:
                    n += 1
                    continue
                try:
                    timestamp = str(df_episode_unix_timestamp.iloc[n].unixtime)
                except:
                    None
                #print('idx :', episode, res.iloc[idx].episode)
                episode_gathered += '/' + res.iloc[idx].episode
                #print(episode_gathered)
                if n % 3 == 0:
                    episode_gathered = res.iloc[idx].episode
                folder = episode_gathered + '/'
                print(timestamp + '|' + 'Rick|A|' + folder)
                #f.write(timestamp + '|' + 'sentiment|A|' + folder + '\n')
                #print('gathered : ', episode_gathered)
                for char in res[res.episode == episode].character:
                    #print(timestamp + '|' + 'sentiment|A|' + add_file)
                    print(timestamp + '|' + 'Rick' + '|' + 'A' + '|' + episode_gathered + '/' + char)
                    f.write(timestamp + '|' + 'Rick' + '|' + 'A' + '|' + episode_gathered + '/' + char + '\n')

                n += 1
                #print(folder + '\n')
                #print('now : ', res.iloc[idx].episode)
                #print('before : ', res.iloc[idx-1].episode)
                #print(episode)


1633992623|Rick|A|Pilot/Bepisian
1633992623|Rick|A|Pilot/Summer Smith
1633992623|Rick|A|Pilot/Pripudlian
1633992623|Rick|A|Pilot/Principal Vagina
1633992623|Rick|A|Pilot/Mrs. Sanchez
1633992623|Rick|A|Pilot/Morty Smith
1633992623|Rick|A|Pilot/Jessica's Friend
1633992623|Rick|A|Pilot/Jessica
1633992623|Rick|A|Pilot/Mr. Goldenfold
1633992623|Rick|A|Pilot/Hookah Alien
1633992623|Rick|A|Pilot/Greebybobe
1633992623|Rick|A|Pilot/Glenn
1633992623|Rick|A|Pilot/Frank Palicky
1633992623|Rick|A|Pilot/Davin
1633992623|Rick|A|Pilot/Canklanker Thom
1633992623|Rick|A|Pilot/Beth Smith
1633992623|Rick|A|Pilot/Jerry Smith
1634079023|Rick|A|Pilot/Anatomy Park/
1634079023|Rick|A|Pilot/Anatomy Park/Joyce Smith
1634079023|Rick|A|Pilot/Anatomy Park/Leonard Smith
1634079023|Rick|A|Pilot/Anatomy Park/Morty Smith
1634079023|Rick|A|Pilot/Anatomy Park/Poncho
1634079023|Rick|A|Pilot/Anatomy Park/Tom Randolph
1634079023|Rick|A|Pilot/Anatomy Park/Ruben
1634079023|Rick|A|Pilot/Anatomy Park/Summer Smith
1634079023|Ric

1636584623|Rick|A|Edge of Tomorty: Rick, Die, Rickpeat/Fascist Shrimp Morty
1636584623|Rick|A|Edge of Tomorty: Rick, Die, Rickpeat/Caterpillar Mr. Goldenfold’s Larvae
1636671023|Rick|A|Edge of Tomorty: Rick, Die, Rickpeat/The Old Man and the Seat/
1636671023|Rick|A|Edge of Tomorty: Rick, Die, Rickpeat/The Old Man and the Seat/Mountain Sweat Jerry
1636671023|Rick|A|Edge of Tomorty: Rick, Die, Rickpeat/The Old Man and the Seat/Vermigurber
1636671023|Rick|A|Edge of Tomorty: Rick, Die, Rickpeat/The Old Man and the Seat/Tony’s Wife
1636671023|Rick|A|Edge of Tomorty: Rick, Die, Rickpeat/The Old Man and the Seat/Tony's Dad
1636671023|Rick|A|Edge of Tomorty: Rick, Die, Rickpeat/The Old Man and the Seat/Tony
1636671023|Rick|A|Edge of Tomorty: Rick, Die, Rickpeat/The Old Man and the Seat/Summer Smith
1636671023|Rick|A|Edge of Tomorty: Rick, Die, Rickpeat/The Old Man and the Seat/Secretary at Tony's
1636671023|Rick|A|Edge of Tomorty: Rick, Die, Rickpeat/The Old Man and the Seat/Running Bird
16366

**Note:**
- Run with
- gource rick_through_episodes.log --background-image /Users/christianr/Documents/DTU/1_semester/02805_Social_graphs_and_interactions/project/wcimages/rick_background.png -1280x720 --seconds-per-day 2 --key --highlight-dirs

- gource rick_through_episodes.log --background-image /Users/christianr/Documents/DTU/1_semester/02805_Social_graphs_and_interactions/project/wcimages/rick_background.png -1280x720 --seconds-per-day 2 --key --highlight-dirs --highlight-user Rick --filename-time 4



# Morty

In [183]:
temp_df = ps.sqldf(
    """
    with jerry_episodes as (
    select 
        'Morty' as char_name
      ,  episode
    from df_characters_exploded
    where lower(name) like '%morty%'
    group by 1, 2
    order by episode
    )
    
    select 
        episode as episode_nr
      , name as character
    from df_characters_exploded df_ex
    where df_ex.episode in (select distinct episode from jerry_episodes)
    and lower(name) not like '%morty%'
    group by 1, 2
    order by 1, 2
    """
)
temp_df

,episode_nr,character
0,1,Bepisian
1,1,Beth Smith
2,1,Canklanker Thom
3,1,Davin
4,1,Frank Palicky
...,...,...
1086,51,Two Crows
1087,51,Unknown Rick
1088,51,Young Beth
1089,51,Young Jerry


In [184]:
res = pd.merge(
    temp_df,
    df_episodes[['name', 'id']],
    how='left',
    left_on='episode_nr',
    right_on='id'
).sort_values(['id'], ascending=True).reset_index(drop=True).rename(columns={'name':'episode'})
res

,episode_nr,character,episode,id
0,1,Bepisian,Pilot,1
1,1,Summer Smith,Pilot,1
2,1,Rick Sanchez,Pilot,1
3,1,Pripudlian,Pilot,1
4,1,Principal Vagina,Pilot,1
...,...,...,...,...
1086,51,Bald Rick,Rickmurai Jack,51
1087,51,Andy,Rickmurai Jack,51
1088,51,Young Jerry,Rickmurai Jack,51
1089,51,Party Rick,Rickmurai Jack,51


In [185]:
n_days = len(res.episode.unique().tolist())

In [186]:
date_range = pd.date_range(datetime.today() - timedelta(days=n_days), periods=n_days)
date_series = pd.Series(date_range)
df_episode_unix_timestamp = pd.DataFrame(pd.DataFrame(date_series))\
    .reset_index(drop=False)\
    .rename(columns={'index':'episode', 0:'timestamp'})
df_episode_unix_timestamp['episode'] = df_episode_unix_timestamp['episode'].apply(lambda x: x+1)
df_episode_unix_timestamp['unixtime'] = df_episode_unix_timestamp['timestamp'].astype(int)
df_episode_unix_timestamp['unixtime'] = (df_episode_unix_timestamp.timestamp - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
#df_episode_unix_timestamp

In [187]:
#res

In [188]:
with open('morty_through_episodes.log', 'w+') as f:
    n = 0
    for idx, row in res.iterrows():
        #print(idx)
        if idx == 0:
            episode = res.iloc[idx].episode
            episode_gathered = res.iloc[idx].episode
            folder = episode_gathered + '/'
            #print(episode)
            #print('idx0 :', episode)
            #print(res[res.episode == episode].character)
            #print(timestamp + '|' + 'sentiment|A|' + folder)
            #f.write(timestamp + '|' + 'sentiment|A|' + folder + '\n')
            timestamp = str(df_episode_unix_timestamp.iloc[n].unixtime)
            #print(timestamp)
            for char in res[res.episode == episode].character:
                #print(episode + '/' + char)
                print(timestamp + '|' + 'Morty' + '|' + 'A' + '|' + episode + '/' + char)
                f.write(timestamp + '|' + 'Morty' + '|' + 'A' + '|' + episode + '/' + char + '\n')


        if idx != 0:
            episode = res.iloc[idx].episode 
            #for char in res[res.episode == episode].character:
            #    print(char)
            if episode != res.iloc[idx - 1].episode:
                episode = res.iloc[idx].episode
                #print(episode)

                if n == 0:
                    n += 1
                    continue
                try:
                    timestamp = str(df_episode_unix_timestamp.iloc[n].unixtime)
                except:
                    None
                #print('idx :', episode, res.iloc[idx].episode)
                episode_gathered += '/' + res.iloc[idx].episode
                #print(episode_gathered)
                if n % 3 == 0:
                    episode_gathered = res.iloc[idx].episode
                folder = episode_gathered + '/'
                print(timestamp + '|' + 'Morty|A|' + folder)
                #f.write(timestamp + '|' + 'sentiment|A|' + folder + '\n')
                #print('gathered : ', episode_gathered)
                for char in res[res.episode == episode].character:
                    #print(timestamp + '|' + 'sentiment|A|' + add_file)
                    print(timestamp + '|' + 'Morty' + '|' + 'A' + '|' + episode_gathered + '/' + char)
                    f.write(timestamp + '|' + 'Morty' + '|' + 'A' + '|' + episode_gathered + '/' + char + '\n')

                n += 1
                #print(folder + '\n')
                #print('now : ', res.iloc[idx].episode)
                #print('before : ', res.iloc[idx-1].episode)
                #print(episode)


1633992625|Morty|A|Pilot/Bepisian
1633992625|Morty|A|Pilot/Summer Smith
1633992625|Morty|A|Pilot/Rick Sanchez
1633992625|Morty|A|Pilot/Pripudlian
1633992625|Morty|A|Pilot/Principal Vagina
1633992625|Morty|A|Pilot/Mr. Goldenfold
1633992625|Morty|A|Pilot/Jessica's Friend
1633992625|Morty|A|Pilot/Jessica
1633992625|Morty|A|Pilot/Mrs. Sanchez
1633992625|Morty|A|Pilot/Hookah Alien
1633992625|Morty|A|Pilot/Greebybobe
1633992625|Morty|A|Pilot/Glenn
1633992625|Morty|A|Pilot/Frank Palicky
1633992625|Morty|A|Pilot/Davin
1633992625|Morty|A|Pilot/Canklanker Thom
1633992625|Morty|A|Pilot/Beth Smith
1633992625|Morty|A|Pilot/Jerry Smith
1634079025|Morty|A|Pilot/Anatomy Park/
1634079025|Morty|A|Pilot/Anatomy Park/Joyce Smith
1634079025|Morty|A|Pilot/Anatomy Park/Leonard Smith
1634079025|Morty|A|Pilot/Anatomy Park/Poncho
1634079025|Morty|A|Pilot/Anatomy Park/Rick Sanchez
1634079025|Morty|A|Pilot/Anatomy Park/Tom Randolph
1634079025|Morty|A|Pilot/Anatomy Park/Ruben
1634079025|Morty|A|Pilot/Anatomy Park/

1636498225|Morty|A|Morty's Mind Blowers/The ABC's of Beth/The Rickchurian Mortydate/
1636498225|Morty|A|Morty's Mind Blowers/The ABC's of Beth/The Rickchurian Mortydate/Presidentress of The Mega Gargantuans
1636498225|Morty|A|Morty's Mind Blowers/The ABC's of Beth/The Rickchurian Mortydate/Summer Smith
1636498225|Morty|A|Morty's Mind Blowers/The ABC's of Beth/The Rickchurian Mortydate/Steve
1636498225|Morty|A|Morty's Mind Blowers/The ABC's of Beth/The Rickchurian Mortydate/Secretary of the Interior
1636498225|Morty|A|Morty's Mind Blowers/The ABC's of Beth/The Rickchurian Mortydate/Rick Sanchez
1636498225|Morty|A|Morty's Mind Blowers/The ABC's of Beth/The Rickchurian Mortydate/President Curtis
1636498225|Morty|A|Morty's Mind Blowers/The ABC's of Beth/The Rickchurian Mortydate/Alien Googah
1636498225|Morty|A|Morty's Mind Blowers/The ABC's of Beth/The Rickchurian Mortydate/Jerry Smith
1636498225|Morty|A|Morty's Mind Blowers/The ABC's of Beth/The Rickchurian Mortydate/Invisi-trooper
163649

**Note:**
- Run with
- gource morty_through_episodes.log --background-image /Users/christianr/Documents/DTU/1_semester/02805_Social_graphs_and_interactions/project/wcimages/morty_background.png -1280x720 --seconds-per-day 2 --key --highlight-dirs

- gource morty_through_episodes.log --background-image /Users/christianr/Documents/DTU/1_semester/02805_Social_graphs_and_interactions/project/wcimages/morty_background.png -1280x720 --seconds-per-day 2 --key --highlight-dirs --highlight-user Morty --filename-time 4



# Summer

In [189]:
temp_df = ps.sqldf(
    """
    with jerry_episodes as (
    select 
        'Summer' as char_name
      ,  episode
    from df_characters_exploded
    where lower(name) like '%summer%'
    group by 1, 2
    order by episode
    )
    
    select 
        episode as episode_nr
      , name as character
    from df_characters_exploded df_ex
    where df_ex.episode in (select distinct episode from jerry_episodes)
    and lower(name) not like '%summer%'
    group by 1, 2
    order by 1, 2
    """
)
temp_df

,episode_nr,character
0,1,Bepisian
1,1,Beth Smith
2,1,Canklanker Thom
3,1,Davin
4,1,Frank Palicky
...,...,...
1044,51,Two Crows
1045,51,Unknown Rick
1046,51,Young Beth
1047,51,Young Jerry


In [190]:
res = pd.merge(
    temp_df,
    df_episodes[['name', 'id']],
    how='left',
    left_on='episode_nr',
    right_on='id'
).sort_values(['id'], ascending=True).reset_index(drop=True).rename(columns={'name':'episode'})
res

,episode_nr,character,episode,id
0,1,Bepisian,Pilot,1
1,1,Rick Sanchez,Pilot,1
2,1,Pripudlian,Pilot,1
3,1,Principal Vagina,Pilot,1
4,1,Mrs. Sanchez,Pilot,1
...,...,...,...,...
1044,51,Diane Sanchez,Rickmurai Jack,51
1045,51,Evil Morty,Rickmurai Jack,51
1046,51,Jerry Smith,Rickmurai Jack,51
1047,51,Beth Smith,Rickmurai Jack,51


In [191]:
n_days = len(res.episode.unique().tolist())

In [192]:
date_range = pd.date_range(datetime.today() - timedelta(days=n_days), periods=n_days)
date_series = pd.Series(date_range)
df_episode_unix_timestamp = pd.DataFrame(pd.DataFrame(date_series))\
    .reset_index(drop=False)\
    .rename(columns={'index':'episode', 0:'timestamp'})
df_episode_unix_timestamp['episode'] = df_episode_unix_timestamp['episode'].apply(lambda x: x+1)
df_episode_unix_timestamp['unixtime'] = df_episode_unix_timestamp['timestamp'].astype(int)
df_episode_unix_timestamp['unixtime'] = (df_episode_unix_timestamp.timestamp - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
#df_episode_unix_timestamp

In [193]:
#res

In [194]:
with open('summer_through_episodes.log', 'w+') as f:
    n = 0
    for idx, row in res.iterrows():
        #print(idx)
        if idx == 0:
            episode = res.iloc[idx].episode
            episode_gathered = res.iloc[idx].episode
            folder = episode_gathered + '/'
            #print(episode)
            #print('idx0 :', episode)
            #print(res[res.episode == episode].character)
            #print(timestamp + '|' + 'sentiment|A|' + folder)
            #f.write(timestamp + '|' + 'sentiment|A|' + folder + '\n')
            timestamp = str(df_episode_unix_timestamp.iloc[n].unixtime)
            #print(timestamp)
            for char in res[res.episode == episode].character:
                #print(episode + '/' + char)
                print(timestamp + '|' + 'Summer' + '|' + 'A' + '|' + episode + '/' + char)
                f.write(timestamp + '|' + 'Summer' + '|' + 'A' + '|' + episode + '/' + char + '\n')


        if idx != 0:
            episode = res.iloc[idx].episode 
            #for char in res[res.episode == episode].character:
            #    print(char)
            if episode != res.iloc[idx - 1].episode:
                episode = res.iloc[idx].episode
                #print(episode)

                if n == 0:
                    n += 1
                    continue
                try:
                    timestamp = str(df_episode_unix_timestamp.iloc[n].unixtime)
                except:
                    None
                #print('idx :', episode, res.iloc[idx].episode)
                episode_gathered += '/' + res.iloc[idx].episode
                #print(episode_gathered)
                if n % 3 == 0:
                    episode_gathered = res.iloc[idx].episode
                folder = episode_gathered + '/'
                print(timestamp + '|' + 'Summer|A|' + folder)
                #f.write(timestamp + '|' + 'sentiment|A|' + folder + '\n')
                #print('gathered : ', episode_gathered)
                for char in res[res.episode == episode].character:
                    #print(timestamp + '|' + 'sentiment|A|' + add_file)
                    print(timestamp + '|' + 'Summer' + '|' + 'A' + '|' + episode_gathered + '/' + char)
                    f.write(timestamp + '|' + 'Summer' + '|' + 'A' + '|' + episode_gathered + '/' + char + '\n')

                n += 1
                #print(folder + '\n')
                #print('now : ', res.iloc[idx].episode)
                #print('before : ', res.iloc[idx-1].episode)
                #print(episode)


1634251835|Summer|A|Pilot/Bepisian
1634251835|Summer|A|Pilot/Rick Sanchez
1634251835|Summer|A|Pilot/Pripudlian
1634251835|Summer|A|Pilot/Principal Vagina
1634251835|Summer|A|Pilot/Mrs. Sanchez
1634251835|Summer|A|Pilot/Morty Smith
1634251835|Summer|A|Pilot/Jessica's Friend
1634251835|Summer|A|Pilot/Jessica
1634251835|Summer|A|Pilot/Mr. Goldenfold
1634251835|Summer|A|Pilot/Hookah Alien
1634251835|Summer|A|Pilot/Greebybobe
1634251835|Summer|A|Pilot/Glenn
1634251835|Summer|A|Pilot/Frank Palicky
1634251835|Summer|A|Pilot/Davin
1634251835|Summer|A|Pilot/Canklanker Thom
1634251835|Summer|A|Pilot/Beth Smith
1634251835|Summer|A|Pilot/Jerry Smith
1634338235|Summer|A|Pilot/Anatomy Park/
1634338235|Summer|A|Pilot/Anatomy Park/Joyce Smith
1634338235|Summer|A|Pilot/Anatomy Park/Leonard Smith
1634338235|Summer|A|Pilot/Anatomy Park/Morty Smith
1634338235|Summer|A|Pilot/Anatomy Park/Poncho
1634338235|Summer|A|Pilot/Anatomy Park/Tom Randolph
1634338235|Summer|A|Pilot/Anatomy Park/Roger
1634338235|Summe

1636930235|Summer|A|One Crew Over the Crewcoo's Morty/Claw and Hoarder: Special Ricktim's Morty/
1636930235|Summer|A|One Crew Over the Crewcoo's Morty/Claw and Hoarder: Special Ricktim's Morty/Talking Cat
1636930235|Summer|A|One Crew Over the Crewcoo's Morty/Claw and Hoarder: Special Ricktim's Morty/The Wizard
1636930235|Summer|A|One Crew Over the Crewcoo's Morty/Claw and Hoarder: Special Ricktim's Morty/Slut Dragon
1636930235|Summer|A|One Crew Over the Crewcoo's Morty/Claw and Hoarder: Special Ricktim's Morty/Shadow Jacker
1636930235|Summer|A|One Crew Over the Crewcoo's Morty/Claw and Hoarder: Special Ricktim's Morty/Rick Sanchez
1636930235|Summer|A|One Crew Over the Crewcoo's Morty/Claw and Hoarder: Special Ricktim's Morty/Morty Smith
1636930235|Summer|A|One Crew Over the Crewcoo's Morty/Claw and Hoarder: Special Ricktim's Morty/Debrah’s Partner
1636930235|Summer|A|One Crew Over the Crewcoo's Morty/Claw and Hoarder: Special Ricktim's Morty/Jerry Smith
1636930235|Summer|A|One Crew Ove

**Note:**
- Run with
- gource summer_through_episodes.log --background-image /Users/christianr/Documents/DTU/1_semester/02805_Social_graphs_and_interactions/project/wcimages/summer_background.png -1280x720 --seconds-per-day 2 --key --highlight-dirs

- gource summer_through_episodes.log --background-image /Users/christianr/Documents/DTU/1_semester/02805_Social_graphs_and_interactions/project/wcimages/summer_background.png -1280x720 --seconds-per-day 2 --key --highlight-dirs --highlight-user Summer --filename-time 4



# Something Else

In [195]:
ps.sqldf(
    """
    select
          replace(df.name, ' ', '_') as character
        , df.episode
        , ts.timestamp
        , ts.unixtime
    from df_characters_exploded df
    inner join df_episode_unix_timestamp as ts
        on ts.episode = df.episode
    order by df.episode asc
    """
)

,character,episode,timestamp,unixtime
0,Rick_Sanchez,1,2021-10-04 15:13:11.890278,1633360391
1,Morty_Smith,1,2021-10-04 15:13:11.890278,1633360391
2,Bepisian,1,2021-10-04 15:13:11.890278,1633360391
3,Beth_Smith,1,2021-10-04 15:13:11.890278,1633360391
4,Canklanker_Thom,1,2021-10-04 15:13:11.890278,1633360391
...,...,...,...,...
1262,Redhead_Morty,51,2021-11-23 15:13:11.890278,1637680391
1263,Young_Jerry,51,2021-11-23 15:13:11.890278,1637680391
1264,Young_Beth,51,2021-11-23 15:13:11.890278,1637680391
1265,Young_Beth,51,2021-11-23 15:13:11.890278,1637680391


`unixtime || '|' || character || '|' || 'A' || '|' || 'episodes/episode_nr/' || episode || '.txt' as output`

In [214]:
# which characters touch which episode
res = ps.sqldf(
    """
    select 
        unixtime || '|' || character || '|' || 'A' || '|' || episode || '.txt' as output
    from
    
    (
    
    select 
       episode
    ,  character
    ,  unixtime
    from 
        (

        select
              replace(df.name, ' ', '_') as character
            , df.episode
            , ts.timestamp
            , ts.unixtime
        from df_characters_exploded df
        inner join df_episode_unix_timestamp as ts
            on ts.episode = df.episode
        order by df.episode asc
        )
    order by episode asc
    )
    """
)
res

,output
0,1633360391|Rick_Sanchez|A|1.txt
1,1633360391|Morty_Smith|A|1.txt
2,1633360391|Bepisian|A|1.txt
3,1633360391|Beth_Smith|A|1.txt
4,1633360391|Canklanker_Thom|A|1.txt
...,...
1262,1637680391|Redhead_Morty|A|51.txt
1263,1637680391|Young_Jerry|A|51.txt
1264,1637680391|Young_Beth|A|51.txt
1265,1637680391|Young_Beth|A|51.txt


In [215]:
#res = ps.sqldf(
#    """
#    
#    select 
#       unixtime || '|' || character || '|' || 'A' || '|' || 'episodes/episode_nr/' || episode || '.txt' as output
#    from 
#    (
#
#    select
#          replace(df.name, ' ', '_') as character
#        , df.episode
#        , ts.timestamp
#        , ts.unixtime
#    from df_characters_exploded df
#    inner join df_episode_unix_timestamp as ts
#        on ts.episode = df.episode
#    )
#    order by 1 desc
#    """
#)

In [219]:
with open('custom.log', 'w') as f:
    for row in res.output:
        #print(row)
        f.write(row + '\n')
        

- gource --hide dirnames,filenames --seconds-per-day 0.1 --auto-skip-seconds 1 -1280x720 -o - | ffmpeg -y -r 60 -f image2pipe -vcodec ppm -i - -vcodec libx264 -preset ultrafast -pix_fmt yuv420p -crf 1 -threads 0 -bf 0 gource.mp4

In [223]:
# which episodes that touch the characters
res = ps.sqldf(
    """
    select 
        unixtime || '|' || episode || '|' || 'A' || '|' || character || '.txt' as output
    from
    
    (
    
    select 
       episode
    ,  character
    ,  unixtime
    from 
        (

        select
              replace(df.name, ' ', '_') as character
            , df.episode
            , ts.timestamp
            , ts.unixtime
        from df_characters_exploded df
        inner join df_episode_unix_timestamp as ts
            on ts.episode = df.episode
        order by df.episode asc
        )
    order by episode asc
    )
    """
)
res

,output
0,1633360391|1|A|Rick_Sanchez.txt
1,1633360391|1|A|Morty_Smith.txt
2,1633360391|1|A|Bepisian.txt
3,1633360391|1|A|Beth_Smith.txt
4,1633360391|1|A|Canklanker_Thom.txt
...,...
1262,1637680391|51|A|Redhead_Morty.txt
1263,1637680391|51|A|Young_Jerry.txt
1264,1637680391|51|A|Young_Beth.txt
1265,1637680391|51|A|Young_Beth.txt


In [224]:
# something
res = ps.sqldf(
    """
    select 
        unixtime || '|' || episode || '|' || 'A' || '|' || character || '/' || episode || '.txt' as output
    from
    
    (
    
    select 
       episode
    ,  character
    ,  unixtime
    from 
        (

        select
              replace(df.name, ' ', '_') as character
            , df.episode
            , ts.timestamp
            , ts.unixtime
        from df_characters_exploded df
        inner join df_episode_unix_timestamp as ts
            on ts.episode = df.episode
        order by df.episode asc
        )
    order by episode asc
    )
    """
)
res

,output
0,1633360391|1|A|Rick_Sanchez/1.txt
1,1633360391|1|A|Morty_Smith/1.txt
2,1633360391|1|A|Bepisian/1.txt
3,1633360391|1|A|Beth_Smith/1.txt
4,1633360391|1|A|Canklanker_Thom/1.txt
...,...
1262,1637680391|51|A|Redhead_Morty/51.txt
1263,1637680391|51|A|Young_Jerry/51.txt
1264,1637680391|51|A|Young_Beth/51.txt
1265,1637680391|51|A|Young_Beth/51.txt
